# Data Preparation

The purpose of this notebook is only to pull from the original json file and export to csv. The intent is to make it easier to do subsequent exploration, visualization, and modeling.

In [64]:
import json
import pandas as pd

In [39]:
reviews = []
for line in open('json/dev/dataset_en_dev.json', 'r'):
    reviews.append(json.loads(line))

In [40]:
len(reviews)

5000

In [26]:
reviews[1]['stars']

'1'

In [30]:
reviews[1]['review_title']

'Garbage!'

In [32]:
reviews[1]['product_category']

'shoes'

In [41]:
reviews[1]['review_body']

"Shoes were purchased on March 6, 2019. My wife wore them a couple times a week since receiving them. April 19, 2019 one of the leather straps broke. Checked return policy and it expired on April 6. One month of warranty? Won't buy shoes from Amazon again."

In [42]:
reviews[1]

{'review_id': 'en_0830781',
 'product_id': 'product_en_0004522',
 'reviewer_id': 'reviewer_en_0731158',
 'stars': '1',
 'review_body': "Shoes were purchased on March 6, 2019. My wife wore them a couple times a week since receiving them. April 19, 2019 one of the leather straps broke. Checked return policy and it expired on April 6. One month of warranty? Won't buy shoes from Amazon again.",
 'review_title': 'Garbage!',
 'language': 'en',
 'product_category': 'shoes'}

In [65]:
# Create individual data columns
def extract_feature(label):
    output = []
    for i in range(0,len(reviews)):
        output.append(reviews[i][label])
    return(pd.DataFrame(output))

# stars = []
# for i in range(0,len(reviews)):
#     stars.append(reviews[i]['stars'])

In [70]:
stars = extract_feature('stars')
title = extract_feature('review_title')
category = extract_feature('product_category')
body = extract_feature('review_body')
reviewer = extract_feature('reviewer_id')

In [69]:
stars.value_counts()

1    1000
2    1000
3    1000
4    1000
5    1000
dtype: int64

Interesting, perfectly balanced category counts...

Is there a possiblity that we have multiple reviews by the same reviewer? If so that might be something to consider, e.g. if there's evidence they're machine generated they might be removed; or if there are many multi-review submissions we can normalize to that reviewer (is it someone who tends to give mostly 1's or mostly 5's?)

In [80]:
reviewer.columns = ['id']

In [118]:
# General form of id itself: remove the leading 'reviewer_en_'
reviewer_id = reviewer['id'].apply(lambda x: x.split('_')[-1])

In [120]:
reviewer_id.value_counts()

0228364    2
0555851    2
0987470    1
0786257    1
0366796    1
          ..
0039415    1
0888331    1
0571450    1
0008837    1
0850986    1
Name: id, Length: 4998, dtype: int64

Meh, only two reviewers in the whole set submitted two reviews each. 

In [ ]:
df = 